In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

import datasets

import numpy as np

Num GPUs Available:  1


In [2]:
def preprocess_image(img):
    width, height = img.size
    target_size = min(width, height)
    left = max(0, (width - target_size ) // 2)
    right = left + target_size
    top = max(0, (height - target_size ) // 2)
    bottom = top + target_size
    crop = img.crop((left, top, right, bottom)).convert('RGB')
    resize = crop.resize((512,512))
    return np.array(resize)

In [3]:
# load image data

ds = datasets.load_from_disk("../../data/diffusion_and_real/")

train_images = np.array([preprocess_image(image) for image in ds["train"]["image"]])
train_labels = np.array(ds["train"]["label"])

vali_images = np.array([preprocess_image(image) for image in ds["vali"]["image"]])
vali_labels = np.array(ds["vali"]["label"])

test_images = np.array([preprocess_image(image) for image in ds["test"]["image"]])
test_labels = np.array(ds["test"]["label"])

In [5]:
train_images.shape

(18000, 512, 512, 3)

In [4]:
train_labels.shape

(18000,)

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(8, (9, 9), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Conv2D(16, (7, 7), strides=(4,4),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Fully connected layers
model.add(layers.Flatten())
# model.add(layers.Dense(10, activation='relu')) removed for now to keep the network simple and fast 
model.add(layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 504, 504, 8)       1952      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 16)        6288      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3600)              0         
                                                                 
 dense (Dense)               (None, 1)                 3

In [7]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 512, 512, 3) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
conv2d (None, 512, 512, 3) float32
max_pooling2d (None, 504, 504, 8) float32
conv2d_1 (None, 126, 126, 8) float32
max_pooling2d_1 (None, 30, 30, 16) float32
flatten (None, 15, 15, 16) float32
dense (None, 3600) float32


[None, None, None, None, None, None]

In [10]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # dear god let this be the right loss function
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=3, batch_size = 32,
                    validation_data=(test_images, test_labels))

: 

: 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


In [1]:
print(test_acc)


NameError: name 'test_acc' is not defined